# Proyecto 2: EDA Ingresos Públicos de Brasil

## 1. Lectura y Exploración Inicial

In [718]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import translators as ts
import os

Cogemos los nombres de los archivos para luego importarlos.

In [719]:
list_df_str = os.listdir(r"datos")[:-1]
print(list_df_str)

['datos-2013.csv', 'datos-2014.csv', 'datos-2015.csv', 'datos-2016.csv', 'datos-2017.csv', 'datos-2018.csv', 'datos-2019.csv', 'datos-2020.csv', 'datos-2021.csv']


Generamos una lista de dataframes.

In [720]:
dfs_initial = list(map(lambda s: pd.read_csv(r"datos/"+s, sep = ";"), list_df_str))

Lista con los años de ejercicio de cada informe.

In [721]:
yearslist = list(map(int, map(lambda x: x.split("-")[1][:4], list_df_str)))
print(yearslist)

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


Creamos una función que nos rellene las filas vacías de los años, teniendo en cuenta que debe tener el valor del ejercicio.

In [722]:
def fill_years(df_list, years_list):
    clean_dfs = []
    i = 0
    for df in df_list:
        df["ANO EXERCÍCIO"] = df["ANO EXERCÍCIO"].fillna(years_list[i])
        clean_dfs.append(df)
        i+=1
    return clean_dfs

Comprobamos que funcione

In [723]:
dfs = fill_years(dfs_initial, yearslist)

In [724]:
for df in dfs:
   print(df["ANO EXERCÍCIO"].unique())

[2013.]
[2014.]
[2015.]
[2016.]
[2017.]
[2018.]
[2019.]
[2020.]
[2021.]


Vamos a comprobar si todos los datos tienen las mismas columnas.

In [725]:
column_checker = []
for i in range(len(dfs)):
    if i == len(dfs)-1:
        break
    column_checker.append(all(dfs[i].columns == dfs[i+1].columns))
all(column_checker)

True

Todos los datos tienen las mismas columnas: ✅

In [726]:
first_datayear = int(list_df_str[0].split("-")[1][:4])

In [727]:
if first_datayear == reduce(lambda x,y: x if np.all(x.dtypes==y.dtypes) else None, dfs)["ANO EXERCÍCIO"].unique():
    print("Los types de los valores son consistentes")

Los types de los valores son consistentes


In [728]:
df_merged = pd.concat(dfs).reset_index(drop = True)

In [729]:
df_merged.columns

Index(['CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR', 'CÓDIGO ÓRGÃO',
       'NOME ÓRGÃO', 'CÓDIGO UNIDADE GESTORA', 'NOME UNIDADE GESTORA',
       'CATEGORIA ECONÔMICA', 'ORIGEM RECEITA', 'ESPÉCIE RECEITA',
       'DETALHAMENTO', 'VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO',
       'VALOR REALIZADO', 'PERCENTUAL REALIZADO', 'DATA LANÇAMENTO',
       'ANO EXERCÍCIO'],
      dtype='object')

In [730]:
newcol = {i:ts.translate_text(i,  to_language = "en").lower().replace(r" ",  r"_") for i in df_merged.columns}
df_merged.rename(columns=newcol, inplace = True)

In [731]:
df_merged.head()

,superior_body_code,name_superior_body,organ_code,organ_name,management_unit_code,name_management_unit,economy_category,source_revenue,recipe_species,detailing,updated_estimated_value,value_launched,realized_value,percentage_realized,release_date,financial_year
0,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",31/12/2013,2013.0
1,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,"0,00","0,00","26666621,42","0,00",31/12/2013,2013.0
2,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,"0,00","0,00","301251,13","0,00",31/12/2013,2013.0
3,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIV.ATIVA POR INFRAÇÃO ADMINISTRATIVA,"0,00","0,00","1855,58","0,00",31/12/2013,2013.0
4,63000.0,Advocacia-Geral da União,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",OUTRAS RESTITUICOES,"0,00","0,00","52140,68","0,00",31/12/2013,2013.0


In [732]:
def dict_codes_gen(code, name):
    dict_codes = dict()
    for i in range(len(code)):
        if pd.isna(code[i]) or pd.isna(name[i]) or code[i] in dict_codes.keys():
            continue
        else:
            dict_codes[code[i]] = name[i]
    return dict_codes

Comprobamos si hay columnas nulas.

In [733]:
print(f"{df_merged.isna().all(axis = 0).sum()} columnas nulas.")

0 columnas nulas.


In [734]:
df_merged.isna().sum()/df_merged.shape[0]

superior_body_code         0.029581
name_superior_body         0.350001
organ_code                 0.024473
organ_name                 0.033993
management_unit_code       0.032720
name_management_unit       0.018982
economy_category           0.018492
source_revenue             0.037434
recipe_species             0.031109
detailing                  0.028585
updated_estimated_value    0.050000
value_launched             0.025742
realized_value             0.038454
percentage_realized        0.023516
release_date               0.023220
financial_year             0.000000
dtype: float64

Creamos diccionarios donde se relacionan el código de una entidad con su nombre.

In [735]:
superior_body_dict = dict_codes_gen(df_merged["superior_body_code"].values, df_merged["name_superior_body"].values)
organ_dict = dict_codes_gen(df_merged["organ_code"].values, df_merged["organ_name"].values)
management_dict = dict_codes_gen(df_merged["management_unit_code"].values, df_merged["name_management_unit"].values)

In [737]:
df_merged.columns

Index(['superior_body_code', 'name_superior_body', 'organ_code', 'organ_name',
       'management_unit_code', 'name_management_unit', 'economy_category',
       'source_revenue', 'recipe_species', 'detailing',
       'updated_estimated_value', 'value_launched', 'realized_value',
       'percentage_realized', 'release_date', 'financial_year'],
      dtype='object')

In [738]:
df_merged.isna().sum()

superior_body_code          30359
name_superior_body         359206
organ_code                  25117
organ_name                  34887
management_unit_code        33581
name_management_unit        19481
economy_category            18978
source_revenue              38418
recipe_species              31927
detailing                   29337
updated_estimated_value     51315
value_launched              26419
realized_value              39465
percentage_realized         24134
release_date                23831
financial_year                  0
dtype: int64

In [739]:
df_merged['organ_name'] = df_merged['organ_code'].map(organ_dict).fillna(df_merged['organ_name'])
df_merged['name_superior_body'] = df_merged['superior_body_code'].map(superior_body_dict).fillna(df_merged['name_superior_body'])
df_merged['name_management_unit'] = df_merged['management_unit_code'].map(management_dict).fillna(df_merged['name_management_unit'])

In [740]:
df_merged.isna().sum()

superior_body_code         30359
name_superior_body         10545
organ_code                 25117
organ_name                   905
management_unit_code       33581
name_management_unit         614
economy_category           18978
source_revenue             38418
recipe_species             31927
detailing                  29337
updated_estimated_value    51315
value_launched             26419
realized_value             39465
percentage_realized        24134
release_date               23831
financial_year                 0
dtype: int64

In [744]:
organ_name_to_code = {v: k for k, v in organ_dict.items()}
superior_body_name_to_code = {v: k for k, v in superior_body_dict.items()}
management_unit_name_to_code = {v: k for k, v in management_dict.items()}

In [747]:
df_merged['organ_code'] = df_merged['organ_name'].map(organ_name_to_code).fillna(df_merged['organ_code'])
df_merged['superior_body_code'] = df_merged['name_superior_body'].map(superior_body_name_to_code).fillna(df_merged['superior_body_code'])
df_merged['management_unit_code'] = df_merged['name_management_unit'].map(management_unit_name_to_code).fillna(df_merged['management_unit_code'])

df_merged.isna().sum()

superior_body_code         10545
name_superior_body         10545
organ_code                   905
organ_name                   905
management_unit_code         614
name_management_unit         614
economy_category           18978
source_revenue             38418
recipe_species             31927
detailing                  29337
updated_estimated_value    51315
value_launched             26419
realized_value             39465
percentage_realized        24134
release_date               23831
financial_year                 0
dtype: int64

In [742]:
df_merged["value_launched"] = df_merged["value_launched"].str.replace(",",".").apply(float).abs()

In [743]:
df_merged.iloc[df_merged["value_launched"].idxmax(axis = 0)]

superior_body_code                                                   25000.0
name_superior_body                                    Ministério da Economia
organ_code                                                           25000.0
organ_name                 Ministério da Economia - Unidades com vínculo ...
management_unit_code                                                170010.0
name_management_unit             SECRETARIA DA RECEITA FEDERAL DO BRASIL-RFB
economy_category                                          Receitas Correntes
source_revenue                   Impostos, Taxas e Contribuições de Melhoria
recipe_species                                                      Impostos
detailing                          PESSOAS JURIDICAS - LIQUIDA DE INCENTIVOS
updated_estimated_value                                                 0,00
value_launched                                           357160677863.969971
realized_value                                                          0,00